# Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from scipy import stats

# Prepare datasets

In [2]:
df_x = pd.read_csv('../data/dataset_01/data.csv')
df_y = pd.read_csv('../data/dataset_01/answer.csv')

In [3]:
# df_x = df_x.drop(labels=['Unnamed: 0'], axis=1)
df_x

In [4]:
# df_y = df_y.drop(labels=['Unnamed: 0'], axis=1)
df_y

In [5]:
df = pd.concat([df_x, df_y], axis=1)
df

In [6]:
labels = [str(i) for i in range(672)]

In [7]:
df = df.drop_duplicates(subset=labels, ignore_index=True)
df

In [8]:
df_x = df.drop(labels=['x1', 'y1', 'z1', 'x2', 'y2', 'z2'], axis=1)
df_x

In [9]:
df_y = df.drop(labels=labels, axis=1)
df_y

In [10]:
df_x = np.reshape(df_x.to_numpy(), (-1, 1, 24, 28))
df_x.shape

In [11]:
df_y = df_y.to_numpy()
df_y

In [12]:
train_x, test_x, train_y, test_y = train_test_split(df_x, df_y, train_size=0.9)

train_x, valid_x, train_y, valid_y = train_test_split(train_x,
                                                    train_y,
                                                    test_size=0.33)

In [13]:
print(train_x.shape), print(train_y.shape)
print(valid_x.shape), print(valid_y.shape)
print(test_x.shape), print(test_y.shape)

In [14]:
train_x = torch.tensor(train_x).float()
train_y = torch.tensor(train_y).float()
valid_x = torch.tensor(valid_x).float()
valid_y = torch.tensor(valid_y).float()
test_x = torch.tensor(test_x).float()
test_y = torch.tensor(test_y).float()

# CNN

In [15]:
lr = 1e-3
batch_size = 64
max_epoch = 30

In [16]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv_1 = nn.Conv2d(1, 32, 7)
        self.max_pool_1 = nn.MaxPool2d(2, stride=2)
        self.conv_2 = nn.Conv2d(32, 64, 3)
        self.max_pool_2 = nn.MaxPool2d(2, stride=2)
        self.hidden_1 = nn.Linear(768, 128)
        self.hidden_2 = nn.Linear(128, 64)
        self.hidden_3 = nn.Linear(64, 64)
        self.hidden_4 = nn.Linear(64, 64)
        self.output = nn.Linear(64, 6)

    def forward(self, x):
        x = torch.relu(self.conv_1(x))
        x = self.max_pool_1(x)
        x = torch.relu(self.conv_2(x))
        x = self.max_pool_2(x)
        x = torch.reshape(x, (x.shape[0], 768))
        x = torch.relu(self.hidden_1(x))
        x = torch.relu(self.hidden_2(x))
        x = torch.relu(self.hidden_3(x))
        x = torch.relu(self.hidden_4(x))
        output = self.output(x)
        return output

In [17]:
model = Model()
model

In [18]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [19]:
# init weights
def init_weights(model) -> None:
    if isinstance(model, nn.Linear):
        torch.nn.init.xavier_uniform_(model.weight)
        model.bias.data.fill_(0.1)

model.apply(init_weights)

# if torch.cuda.is_available():
#     model = model.cuda()

#     train_x = train_x.cuda()
#     train_y = train_y.cuda()
#     valid_x = valid_x.cuda()
#     valid_y = valid_y.cuda()
#     test_x = test_x.cuda()
#     test_y = test_y.cuda()

In [20]:
losses_train = []
losses_valid = []

for epoch in range(max_epoch):
    train_loss = 0.0
    for i in range(0, train_x.size()[0], batch_size):
        data = train_x[i:i + batch_size]
        target = train_y[i:i + batch_size]

        optimizer.zero_grad()
        prediction = model(data)
        loss = loss_function(prediction, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    valid_loss = 0.0
    for i in range(0, valid_x.size()[0], batch_size):
        data = valid_x[i:i + batch_size]
        target = valid_y[i:i + batch_size]

        prediction = model(data)
        loss = loss_function(prediction, target)
        valid_loss += loss.item()

    # if (epoch) % 10 == 0:
    print(
        f'Epoch {epoch} \t\t Training Loss: {train_loss / len(train_x)} \t\t Validation Loss: {valid_loss / len(valid_x)}'
    )
    
    if (epoch > 0 and valid_loss / len(valid_x)  < min(losses_valid)):
        torch.save(model.state_dict(), 
           "../logs/weights_model_03/model_state_adam_01.pt")
        print("Save model")
        
    
    losses_train.append(train_loss / len(train_x))
    losses_valid.append(valid_loss / len(valid_x))

In [113]:
# torch.save(model.state_dict(), 
        #    "../logs/weights_model_03/model_state_rmsprop.pt")

In [21]:
losses = pd.DataFrame([losses_train, losses_valid],
                      index=["Train loss", "Valid loss"]).T
losses

In [22]:
ax = losses.plot(colormap="brg")
ax.set_xlabel("Номер эпохи")
ax.set_ylabel("Значение функции потерь")

In [17]:
# Генерируем выборку с распределением Рэлея
sample_size = 8000
scale_parameter = 0.15
rayleigh_sample = np.random.rayleigh(scale_parameter, sample_size)

# Визуализация выборки
plt.hist(rayleigh_sample, bins=50, density=True, alpha=0.7, color='b', edgecolor='black')
plt.title('Распределение угла')
plt.xlabel('Угол, радианы')
plt.ylabel('Плотность распределения')
plt.show()

In [18]:
rayleigh_sample.mean()

In [19]:
rayleigh_sample.std()

In [20]:
np.quantile(rayleigh_sample, 0.68)

In [5]:
import numpy as np
import matplotlib.pyplot as plt

# Задаем несколько параметров масштаба
scale_parameters = 0.14

# Генерируем выборки с разными параметрами масштаба
sample_size = 8000
rayleigh_samples = np.random.rayleigh(scale_parameters, sample_size)

# Визуализация выборок
plt.hist(rayleigh_samples, bins=50, density=True, alpha=0.5, label=f'Распределение Рэлея')
plt.hist(rayleigh_sample, bins=50, density=True, alpha=0.5, label=f'Угловое распределение')

plt.title('Сравнение распределений')
plt.xlabel('Угол, радианы')
plt.ylabel('Плотность распределения')
plt.legend()
plt.show()

In [27]:
plt.boxplot([rayleigh_sample, rayleigh_samples], labels=[f'Угловое распределение', f'Распределение Рэлея'])

plt.title('Сравнение распределений')
plt.ylabel('Уголовое распределение, радианы')
plt.legend()
plt.show()